In [ ]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

In [ ]:
llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)

cache_dir = LocalFileStore(".cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter3.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

vectorstore = FAISS.from_documents(
    docs,
    cached_embeddings,
)

retriver = vectorstore.as_retriever()

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 아주 도움이되는 비서입니다. 당신은 다음의 context만 사용해서 대답해야합니다. 만약 당신이 질문에 대한 답을 모른다면 절대로 추측하지 말고 반드시 모른다고 얘기하세요.\n\n{context}",
        ),
        MessagesPlaceholder(
            variable_name="history",
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

In [ ]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = (
    (
        {
            "context": retriver,
            "question": RunnablePassthrough(),
        }
    )
    | RunnablePassthrough.assign(history=load_memory)
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)

In [ ]:
invoke_chain("Aaronson은 유죄인가요?")

In [ ]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

In [ ]:
invoke_chain("Julia 는 누구인가요?")

In [ ]:
load_memory({})

In [ ]:
invoke_chain("내가 처음에 한 질문은?")

In [ ]:
invoke_chain("Aaronson 는 누구인가요?")

In [ ]:
invoke_chain("Aaronson은 유죄인가요?")

In [ ]:
invoke_chain("마지막 질문은?")